In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import torch

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
     |████████████████████████████████| 880 kB 43.2 MB/s 
     |████████████████████████████████| 6.6 MB 35.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e747c2c1fa71454c8135ccd85d332fdfdf7baa8136e3e240ec97a91975cc0e20
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Preprocessing

In [ ]:
# train labels
train_label_file = open('./train.label.txt', 'r') 
train_labels = train_label_file.readlines()
train_labels = [label.strip('\n') for label in train_labels]

#dev labels
dev_label_file = open('./dev.label.txt', 'r')
dev_labels = dev_label_file.readlines()
dev_labels = [label.strip('\n') for label in dev_labels]

In [ ]:
# train tweets
f = open(f'./tweet_text.pckl','rb')
train_data = pickle.load(f)
f.close()

# dev tweets
f = open(f'./dev_tweet_text.pckl','rb')
dev_data = pickle.load(f)
f.close()

In [8]:
## cleaning the tweets
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text) #remove @mention
    text = re.sub(r'#','',text) # remove the hashtag symbol
    text = re.sub(r'https?:\/\/\S+', '',text) #remove hyperlink
    text = re.sub(r'\n','',text) # remove \n 
    text = re.sub(r'\r','',text) # remove \r
    text = re.sub(r'[0-9]+','',text) #remove all the number
    text = re.sub(r'\W+', ' ', text) #remove special characters
    text = text.strip().lower()
    if len(text) != 0:
        return text
    else:
        return None

In [ ]:
for i in range(len(train_data)):
    for j in range(len(train_data[i])):
        train_data[i][j] = clean_text(train_data[i][j])
    train_data[i] = [x for x in train_data[i] if x is not None]
        
for i in range(len(dev_data)):
    for j in range(len(dev_data[i])):
        dev_data[i][j] = clean_text(dev_data[i][j])
    dev_data[i] = [x for x in dev_data[i] if x is not None]

In [ ]:
train_data[0]

['can regularly rinsing your nose with saline help prevent infection with the new coronavirus',
 'can eating garlic help prevent infection with the new coronavirus covidmalaysia',
 'do vaccines against pneumonia protect you against the new coronavirus',
 'can spraying alcohol or chlorine all over your body kill the new coronavirus chamber',
 'how effective are thermal scanners in detecting people infected with the new coronavirus',
 'can an ultraviolet disinfection lamp kill the new coronavirus',
 'are hand dryers effective in killing the new coronavirus',
 'the new coronavirus cannot be transmitted through mosquito bites',
 'taking a hot bath does not prevent the new coronavirus disease',
 'cold weather and snow cannot kill the new coronavirus',
 'covid virus can be transmitted in areas with hot and humid climates',
 'drinking alcohol does not protect you against covid and can be dangerous',
 'being able to hold your breath for seconds or more without coughing or feeling discomfort do

In [6]:
maxlen = 512

def split_based_length(text, maxlen):
  ts = list(filter(None, text.split(" ")))
  split_text = []
  #print("lents", len(ts))
  while len(ts) > maxlen:
    split_text.append(ts[:maxlen])
    ts = ts[maxlen:]
  if len(ts) != 0:
    split_text.append(ts)
    #print("leftts", ts)
  return [" ".join(word for word in s) for s in split_text]

In [7]:
# merge source tweeet and reply tweet together for train data
train_merge_labels=[]
train_merge_events=[]
for event in train_data:
  merge = ' '.join(word for word in event)
  split_text = split_based_length(merge, maxlen)
  for text in split_text:
    train_merge_events.append(str(text))
    train_merge_labels.append(train_labels[i])

# merge source tweeet and reply tweet together for dev data
dev_merge_events=[]
dev_merge_labels=[]
for event in dev_data:
  merge = ' '.join(word for word in event)
  split_text = split_based_length(merge, maxlen)
  for text in split_text:
    dev_merge_events.append(str(text))
    dev_merge_labels.append(dev_labels[i])

NameError: ignored

In [ ]:
train_merge_events[0]

'can regularly rinsing your nose with saline help prevent infection with the new coronavirus can eating garlic help prevent infection with the new coronavirus covidmalaysia do vaccines against pneumonia protect you against the new coronavirus can spraying alcohol or chlorine all over your body kill the new coronavirus chamber how effective are thermal scanners in detecting people infected with the new coronavirus can an ultraviolet disinfection lamp kill the new coronavirus are hand dryers effective in killing the new coronavirus the new coronavirus cannot be transmitted through mosquito bites taking a hot bath does not prevent the new coronavirus disease cold weather and snow cannot kill the new coronavirus covid virus can be transmitted in areas with hot and humid climates drinking alcohol does not protect you against covid and can be dangerous being able to hold your breath for seconds or more without coughing or feeling discomfort does not mean you are free from the coronavirus dis

In [ ]:
train_df = pd.DataFrame({'text':train_merge_events, 'label':train_merge_labels})
train_df['label'] = LabelEncoder().fit_transform(train_df['label'])
nan_value = float("NaN")
train_df.replace("", nan_value, inplace=True)
train_df.dropna(axis=0,inplace=True)
print("train", train_df.shape)
train_df.to_csv('./t_hug.tsv', sep='\t',index=False)

dev_df = pd.DataFrame({'text':dev_merge_events, 'label':dev_merge_labels})
dev_df['label'] = LabelEncoder().fit_transform(dev_df['label'])
dev_df.replace("", nan_value, inplace=True)
dev_df.dropna(axis=0,inplace=True)
print("dev", dev_df.shape)
dev_df.to_csv('./d_hug.tsv', sep='\t',index=False)

train (1985, 2)
dev (661, 2)


In [ ]:
train_df

,text,label
0,can regularly rinsing your nose with saline he...,0
1,french police chief killed himself after charl...,0
2,coronavirus disease covid advice for the publi...,0
3,ottawa police confirm that there were multiple...,0
4,if the primary focus of a government isn t to ...,0
...,...,...
1980,desperate ted cruz claims planned parenthood s...,0
1981,thoughts and prayers are not enough pres obama...,0
1982,police have surrounded this building where the...,0
1983,joseph smith who translated it by the gift amp...,0


# Training

In [9]:
from transformers import BertModel
from transformers import BertTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification
import torch.nn as nn
import torch.optim as optim
import time


In [12]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

#load BERT's WordPiece tokenisation model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:

#from transformers import AutoTokenizer

class SSTDataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']
        label = self.df.loc[index, 'label']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [14]:

train_set = SSTDataset(filename ='./t_hug.tsv', maxlen = 512)
dev_set = SSTDataset(filename ='./d_hug.tsv', maxlen = 512)

train_loader = DataLoader(train_set,batch_size = 8, num_workers = 0)
dev_loader = DataLoader(dev_set, batch_size = 8, num_workers = 0)

print("Done preprocessing training and development data.")

Done preprocessing training and development data.


In [22]:
class SentimentClassifier(nn.Module):
    def __init__(self):
        super(SentimentClassifier, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        self.cls_layer = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        outputs = self.bert_layer(seq, attention_mask = attn_masks, return_dict=True)
        cont_reps = outputs.last_hidden_state
        cls_rep = cont_reps[:, 0]
        logits = self.cls_layer(cls_rep)
        return logits

In [23]:
gpu = 0 #gpu ID

print("Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...")
net = SentimentClassifier()
net.cuda(gpu) #Enable gpu support for the model
print("Done creating the sentiment classifier.")

Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done creating the sentiment classifier.


In [32]:
criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [19]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [20]:
def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))

In [21]:
num_epoch = 1

#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

Iteration 0 of epoch 0 complete. Loss: 0.6581687927246094; Accuracy: 0.625; Time taken (s): 1.6062862873077393
Iteration 100 of epoch 0 complete. Loss: 0.0005389024736359715; Accuracy: 1.0; Time taken (s): 147.1701259613037
Iteration 200 of epoch 0 complete. Loss: 0.0002770781866274774; Accuracy: 1.0; Time taken (s): 146.67042994499207
Epoch 0 complete! Development Accuracy: 0.9999999403953552; Development Loss: 0.0001617143172925024
Best development accuracy improved from 0 to 0.9999999403953552, saving model...


# Testing

In [12]:
f = open(f'/test_tweet_text.pckl','rb')
test_data = pickle.load(f)
f.close()

In [13]:
for i in range(len(test_data)):
    for j in range(len(test_data[i])):
        test_data[i][j] = clean_text(test_data[i][j])
    test_data[i] = [x for x in test_data[i] if x is not None]

test_merge_events=[]
test_merge_index=[]
for i, event in enumerate(test_data):
  merge = ' '.join(word for word in event)
  split_text = split_based_length(merge, maxlen)
  for text in split_text:
    test_merge_events.append(str(text))
    test_merge_index.append(i)

In [17]:
test_df = pd.DataFrame({'text':test_merge_events, 'index': test_merge_index})
nan_value = float("NaN")
test_df.replace("", nan_value, inplace=True)
test_df.dropna(axis=0,inplace=True)
print("test", test_df.shape)
test_df.to_csv('/content/ts_hug.tsv', sep='\t',index=False)

test (615, 2)


In [31]:
class TestDataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']
        index = self.df.loc[index, 'index']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, index

In [32]:
test_set = TestDataset(filename = '/content/ts_hug.tsv', maxlen = 512)

#Creating intsances of training and development dataloaders
test_loader = DataLoader(test_set,batch_size = 1, num_workers = 0)

In [81]:
def get_preds(probs, preds, ix):
  prev = 0
  count_1 = count_0 = probs_1 = probs_0 = 0
  predictions = []
  l = len(ix)-1
  for i, (po, pd, ix) in enumerate(zip(probs, preds, ix)):
    # print("ix, prev: ", ix, prev)
    # print("po pd", po, pd)
    # print(ix != prev)
    ## when move to the next events or when reach the last text
    if ((ix != prev) | (i == l)):
      # print("count", count_1, count_0)
      # print("prob", probs_1, probs_0)
      if count_1 > count_0:
        predictions.append(1)
      elif count_0 > count_1:
        predictions.append(0)
      else:
        if probs_1 >= probs_0:
          predictions.append(1)
        else:
          predictions.append(0)
      count_1 = count_0 = probs_1 = probs_0 = 0
    prev = ix
    if pd == 1:
      probs_1+= po
      count_1+=1
    else:
      probs_0+= po
      count_0+=1
    # print("count probs: ", count_1, count_0, probs_1, probs_0, prev)
    # print(predictions)
  return predictions

def predict(net,test_loader,weight_file):
  # load weight
  net.load_state_dict(torch.load(weight_file))
  probs_ = []
  preds_ = []
  ix_ = []
  predictions = []

  # a = True
  
  # Predict process
  with torch.no_grad():
      for seq, attn_masks, index in test_loader:
          seq, attn_masks, index = seq.cuda(gpu), attn_masks.cuda(gpu), index.cuda(gpu)
          # logits, train_out = net(seq, attn_masks)
          logits = net(seq, attn_masks)
          probs = torch.sigmoid(logits.unsqueeze(-1))
          soft_probs = (probs > 0.5).long()
          # predictions.append(soft_probs.cpu().numpy().squeeze())
          probs_.append(probs.cpu().numpy().squeeze())
          preds_.append(soft_probs.cpu().numpy().squeeze())
          ix_.append(index.cpu().numpy().squeeze())

          # if a:
          #   print(soft_probs)
          #   print(probs.cpu().numpy().squeeze())
          #   print(index.cpu().numpy().squeeze())
          #   a= False

  predictions = get_preds(probs_, preds_, ix_)
  return predictions

In [82]:
weight_file = "/content/drive/MyDrive/colab_info/sstcls_0.dat"
prediction = predict(net, test_loader, weight_file)

In [83]:
prediction

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [84]:
df = pd.DataFrame({"Id": range(len(prediction)),"Predicted": prediction}) 
df.to_csv('/content/p_hug.csv',index=False)